In [0]:
!unzip "/content/drive/My Drive/glove.6B"

Archive:  /content/drive/My Drive/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import numpy as np
import os
import pandas as pd

In [0]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
type(embeddings_index)

dict

In [0]:
print(embeddings_index["dog"])

[-1.1043e-01  8.1217e-01  7.3668e-02  1.9023e-01 -5.2888e-02  6.1468e-02
  1.6076e-01  4.1302e-01 -3.0199e-01 -9.0827e-01  2.7504e-01 -3.1890e-02
 -2.8842e-01  2.3447e-01  4.7679e-01  5.0124e-01  2.9371e-01  2.7029e-01
  5.4745e-02  9.8038e-02  5.7116e-01  3.6755e-01  4.0734e-02  3.4347e-01
 -1.8256e-01 -2.8935e-01  2.3826e-02 -1.9401e-01  2.4444e-01  1.3407e-01
 -1.6494e-01 -2.6983e-01 -2.6234e-01 -2.1779e-01 -8.7528e-01  7.3822e-01
 -8.7931e-02 -1.0876e-02 -2.6540e-01  3.4668e-01 -5.5814e-01  1.7591e-01
  1.6926e-01 -1.5725e-01 -5.0430e-01 -2.0100e-01  6.6701e-01 -3.2518e-02
  4.5012e-02  6.5675e-02 -1.6061e-01 -7.3363e-01  2.4642e-01  3.4325e-01
  2.1899e-01  4.8646e-02 -5.9987e-01 -5.8153e-02 -5.1694e-02 -5.7846e-01
  3.0000e-01  3.5078e-01  4.6646e-01 -7.5309e-03  1.0455e-01 -5.1016e-01
 -5.5987e-02 -1.0295e-01 -2.6476e-01 -4.1230e-02 -2.8371e-02  5.1979e-01
 -3.4849e-01 -4.7217e-01 -3.7229e-01 -3.2790e-02  1.3989e-01  3.5716e-01
  1.9305e-01 -2.1986e-01  2.4136e-01  4.0976e-01  3

In [0]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=28fe3c83deefcfdfa6e455ca4be1a06de743630f8e766876398e809c578ca291
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [0]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')

In [0]:
len(en_stop)

174

In [0]:
df=pd.read_pickle("/content/drive/My Drive/pinky_promise_last.pckl")

In [0]:
from ast import literal_eval
df.genre_ids=df.genre_ids.apply(literal_eval)

In [0]:
movie_mean_wordvec=np.zeros((len(df),300))
movie_mean_wordvec.shape

(42978, 300)

In [0]:
genres=[]
rows_to_delete=[]
for i in range(len(df)):
    movie_genres=df.genre_ids[i]
    genres.append(movie_genres)
    overview=df.overview[i]
    tokens = tokenizer.tokenize(overview)
    stopped_tokens = [k for k in tokens if not k in en_stop]
    count_in_vocab=0
    s=0
    if len(stopped_tokens)==0:
        rows_to_delete.append(i)
        genres.pop(-1)
#         print(overview)
#         print("sample ",i,"had no nonstops")
    else:
        for tok in stopped_tokens:
            if tok.lower() in embeddings_index:
                count_in_vocab+=1
                s+=embeddings_index[tok.lower()]
        if count_in_vocab!=0:
            movie_mean_wordvec[i]=s/float(count_in_vocab)
        else:
            rows_to_delete.append(i)
            genres.pop(-1)
#             print(overview)
#             print("sample ",i,"had no word2vec")

In [0]:
mask2=[]
for row in range(len(movie_mean_wordvec)):
    if row in rows_to_delete:
        mask2.append(False)
    else:
        mask2.append(True)

In [0]:
X=movie_mean_wordvec[mask2]

In [0]:
X.shape

(42978, 300)

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

Y=mlb.fit_transform(genres)
Y.shape

(42978, 14)

In [0]:
type(genres)

list

In [0]:
list(mlb.classes_)

[' ', ',', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']']

In [0]:
(X,Y)=glove_features

print(X.shape)
print(Y.shape)

(42978, 300)
(42978, 14)


In [0]:
import numpy as np
mask_text=np.random.rand(len(X))<0.8

X_train=X[mask_text]
Y_train=Y[mask_text]
X_test=X[~mask_text]
Y_test=Y[~mask_text]

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model_textual = Sequential([
    Dense(300, input_shape=(300,)),
    Activation('relu'),
    Dense(14),
    Activation('softmax'),
])

model_textual.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model_textual.fit(X_train, Y_train, epochs=10000, batch_size=500, verbose=0 )

In [0]:
score = model_textual.evaluate(X_test, Y_test, batch_size=249)
print(score)

8622/8622 [==============================] - 0s 14us/step
[1.8184351127182516, 0.45441895723342896]


In [0]:
print("%s: %.2f%%" % (model_textual.metrics_names[1], score[1]*100))

accuracy: 45.44%


In [0]:
Y_preds=model_textual.predict(X_test)

In [0]:
!pip install tmdbsimple
import tmdbsimple as tmdb

In [0]:
api_key = "d0ccc864ded48afb6a7e28b2d32001ed"
tmdb.API_KEY = api_key
genres=tmdb.Genres()
list_of_genres=genres.movie_list()['genres']
Genre_ID_to_name={}
for i in range(len(list_of_genres)):
  genre_id=list_of_genres[i]['id']
  genre_name=list_of_genres[i]['name']
  Genre_ID_to_name[genre_id]=genre_name

In [0]:
genre_list=sorted(list(Genre_ID_to_name.keys()))

In [0]:
def precision_recall(gt,preds):
   TP=0
   FP=0
   FN=0
   for t in gt:
       if t in preds:
           TP+=1
       else:
           FN+=1
   for p in preds:
       if p not in gt:
           FP+=1
   if TP+FP==0:
       precision=0
   else:
       precision=TP/float(TP+FP)
   if TP+FN==0:
       recall=0
   else:
       recall=TP/float(TP+FN)
   return precision,recall

In [0]:
print("Our predictions for the movies are - \n")
precs=[]
recs=[]
for i in range(len(Y_preds)):
    row=Y_preds[i]
    gt_genres=Y_test[i]
    gt_genre_names=[]
    for j in range(14):
        if gt_genres[j]==1:
            gt_genre_names.append(Genre_ID_to_name[genre_list[j]])
    top_3=np.argsort(row)[-3:]
    predicted_genres=[]
    for genre in top_3:
        predicted_genres.append(Genre_ID_to_name[genre_list[genre]])
    (precision,recall)=precision_recall(gt_genre_names,predicted_genres)
    precs.append(precision)
    recs.append(recall)
    if i%50==0:
        print("Predicted: ",predicted_genres," Actual: ",gt_genre_names)

Our predictions for the movies are - 

Predicted:  ['History', 'Action', 'Documentary']  Actual:  ['Adventure', 'Fantasy', 'Thriller', 'Crime', 'Documentary', 'Science Fiction', 'Mystery']
Predicted:  ['Fantasy', 'Adventure', 'Western']  Actual:  ['Adventure', 'Fantasy', 'Animation', 'Drama', 'History', 'Western', 'Thriller', 'Science Fiction', 'Mystery']
Predicted:  ['Mystery', 'History', 'Action']  Actual:  ['Adventure', 'Fantasy', 'Horror', 'Action', 'History', 'Crime', 'Science Fiction', 'Mystery']
Predicted:  ['Horror', 'History', 'Action']  Actual:  ['Adventure', 'Fantasy', 'Horror', 'Action', 'Comedy', 'History', 'Western', 'Crime', 'Documentary', 'Science Fiction', 'Mystery']
Predicted:  ['Drama', 'Mystery', 'Science Fiction']  Actual:  ['Documentary', 'Science Fiction', 'Mystery']
Predicted:  ['Drama', 'Science Fiction', 'Mystery']  Actual:  ['Adventure', 'Fantasy', 'Animation', 'Drama', 'Action', 'Comedy', 'History', 'Western', 'Thriller', 'Crime', 'Documentary', 'Science Fic

In [0]:
print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("avg=",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

0.8037578288100208 0.3562116145536465
avg= 0.5799847216818337


In [0]:
print("f1-score", (2*(np.mean(np.asarray(precs)) * np.mean(np.asarray(recs)))) / (np.mean(np.asarray(precs)) + np.mean(np.asarray(recs))))

f1-score 0.49364726898377315
